# Segmentation

This notebook is for segmenting timelapse microscopy data, with associated sinhgle-cell labels and tracks, showing the infection of human macrophages with Mycobacterium Tuberculosis (Mtb), acquired on an Opera Phenix confocal microscope. 

In [47]:
import napari
import cellpose
from octopuslite import utils, tile
import numpy as np

def view(img):
    return napari.Viewer().add_image(img)

### Load experiment of choice

The Opera Phenix is a high-throughput confocal microscope that acquires very large 5-dimensional (TCZXY) images over several fields of view in any one experiment. Therefore, a lazy-loading approach is chosen to mosaic, view and annotate these images. This approach depends upon Dask and DaskFusion. The first step is to load the main metadata file (typically called `Index.idx.xml` and located in the main `Images` directory) that contains the image filenames and associated TCXZY information used to organise the images.

In [3]:
image_dir = '/mnt/DATA/sandbox/pierre_live_cell_data/outputs/Replication_IPSDM_GFP/Images/'
metadata_fn = '/mnt/DATA/sandbox/pierre_live_cell_data/outputs/Replication_IPSDM_GFP/Index.idx.xml'
metadata = utils.read_harmony_metadata(metadata_fn)

Reading metadata XML file...


Extracting HarmonyV5 metadata:   0%|          | 0/113400 [00:00<?, ?it/s]

Extracting metadata complete!


In [5]:
metadata

,id,State,URL,Row,Col,FieldID,PlaneID,TimepointID,ChannelID,FlimID,...,PositionZ,AbsPositionZ,MeasurementTimeOffset,AbsTime,MainExcitationWavelength,MainEmissionWavelength,ObjectiveMagnification,ObjectiveNA,ExposureTime,OrientationMatrix
0,0303K1F1P1R1,Ok,r03c03f01p01-ch1sk1fk1fl1.tiff,3,3,1,1,0,1,1,...,0,0.135583505,0,2021-04-16T19:09:33.84+01:00,488,522,40,1.1,0.1,"[[0.990860,0,0,-15.9],[0,-0.990860,0,-44.8],[0..."
1,0303K1F1P1R2,Ok,r03c03f01p01-ch2sk1fk1fl1.tiff,3,3,1,1,0,2,1,...,0,0.135583505,0,2021-04-16T19:09:33.84+01:00,640,706,40,1.1,0.2,"[[0.990860,0,0,-15.9],[0,-0.990860,0,-44.8],[0..."
2,0303K1F1P2R1,Ok,r03c03f01p02-ch1sk1fk1fl1.tiff,3,3,1,2,0,1,1,...,2E-06,0.135585502,0,2021-04-16T19:09:34.12+01:00,488,522,40,1.1,0.1,"[[0.990860,0,0,-15.9],[0,-0.990860,0,-44.8],[0..."
3,0303K1F1P2R2,Ok,r03c03f01p02-ch2sk1fk1fl1.tiff,3,3,1,2,0,2,1,...,2E-06,0.135585502,0,2021-04-16T19:09:34.12+01:00,640,706,40,1.1,0.2,"[[0.990860,0,0,-15.9],[0,-0.990860,0,-44.8],[0..."
4,0303K1F1P3R1,Ok,r03c03f01p03-ch1sk1fk1fl1.tiff,3,3,1,3,0,1,1,...,4E-06,0.135587499,0,2021-04-16T19:09:34.4+01:00,488,522,40,1.1,0.1,"[[0.990860,0,0,-15.9],[0,-0.990860,0,-44.8],[0..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
113395,0609K75F9P1R2,Ok,r06c09f09p01-ch2sk75fk1fl1.tiff,6,9,9,1,74,2,1,...,0,0.135533601,266399.61,2021-04-19T21:14:19.477+01:00,640,706,40,1.1,0.2,"[[0.990860,0,0,-15.9],[0,-0.990860,0,-44.8],[0..."
113396,0609K75F9P2R1,Ok,r06c09f09p02-ch1sk75fk1fl1.tiff,6,9,9,2,74,1,1,...,2E-06,0.135535598,266399.61,2021-04-19T21:14:19.757+01:00,488,522,40,1.1,0.1,"[[0.990860,0,0,-15.9],[0,-0.990860,0,-44.8],[0..."
113397,0609K75F9P2R2,Ok,r06c09f09p02-ch2sk75fk1fl1.tiff,6,9,9,2,74,2,1,...,2E-06,0.135535598,266399.61,2021-04-19T21:14:19.757+01:00,640,706,40,1.1,0.2,"[[0.990860,0,0,-15.9],[0,-0.990860,0,-44.8],[0..."
113398,0609K75F9P3R1,Ok,r06c09f09p03-ch1sk75fk1fl1.tiff,6,9,9,3,74,1,1,...,4E-06,0.135537595,266399.61,2021-04-19T21:14:20.037+01:00,488,522,40,1.1,0.1,"[[0.990860,0,0,-15.9],[0,-0.990860,0,-44.8],[0..."


### View assay layout and mask information (optional)

The Opera Phenix acquires many time lapse series from a range of positions. The first step is to inspect the image metadata, presented in the form of an `Assaylayout/experiment_ID.xml` file, to show which positions correspond to which experimental assays.

In [6]:
metadata_path = '/mnt/DATA/sandbox/pierre_live_cell_data/outputs/Replication_IPSDM_GFP/Assaylayout/20210602_Live_cell_IPSDMGFP_ATB.xml'
utils.read_harmony_metadata(metadata_path, assay_layout=True)

Reading metadata XML file...
Extracting metadata complete!


Strain Compound Concentration ConcentrationEC
3 4    RD1     CTRL             0             EC0
  5     WT     CTRL             0             EC0
  6     WT      PZA            60            EC50
  7     WT      RIF           0.1            EC50
  8     WT      INH          0.04            EC50
  9     WT      BDQ          0.02            EC50
4 4    RD1     CTRL             0             EC0
  5     WT     CTRL             0             EC0
  6     WT      PZA            60            EC50
  7     WT      RIF           0.1            EC50
  8     WT      INH          0.04            EC50
  9     WT      BDQ          0.02            EC50
5 4    RD1     CTRL             0             EC0
  5     WT     CTRL             0             EC0
  6     WT      PZA           400            EC99
  7     WT      RIF             2            EC99
  8     WT      INH             2            EC99
  9     WT      BDQ           2.5            EC99
6 4    RD1     CTRL             0             EC0
  5     WT     CTRL             0             EC0
  6     WT      PZA           400            EC99
  7     WT      RIF             2            EC99
  8     WT      INH             2            EC99
  9     WT      BDQ           2.5            EC99

### Define row and column of choice

In [7]:
row = '6'
column = '9'

### Now to lazily mosaic the images using Dask prior to viewing them.

1x (75,2,3) [TCZ] image stack takes approximately 1 minute to stitch together, so only load the one field of view I want.

In [49]:
rfp_images = tile.compile_mosaic(image_dir, 
                             metadata, 
                             row, 
                             column, 
                             set_channel=2, 
                             set_plane = 'sum_proj',
#                              set_time = 1
                         )#.astype(uint8)
rfp = rfp_images[0].compute().compute().astype(np.uint16)

In [52]:
gfp_images = tile.compile_mosaic(image_dir, 
                             metadata, 
                             row, 
                             column, 
                             set_channel=1, 
                             set_plane = 'sum_proj',
#                              set_time = 1
                         )#.astype(uint8)
gfp = gfp_images[0].compute().compute().astype(np.uint16)

# Segment 
Let us start simple, only segmenting the lowest Z plane where the largest regions of cells are and only ch1 (GFP) where the GFP signal is.

In [113]:
!nvcc --version
!nvidia-smi

from cellpose import core, utils, io, models, metrics

use_GPU = core.use_gpu()
yn = ['NO', 'YES']
print(f'>>> GPU activated? {yn[use_GPU]}')

model = models.Cellpose(gpu=True, model_type='cyto')

def segment(img):
    masks, flows, styles, diams = model.eval(img, diameter=250, channels=[0,0],
                                             flow_threshold=None, cellprob_threshold=0)
    return masks

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2019 NVIDIA Corporation
Built on Sun_Jul_28_19:07:16_PDT_2019
Cuda compilation tools, release 10.1, V10.1.243
Thu Jan 19 12:03:44 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 515.86.01    Driver Version: 515.86.01    CUDA Version: 11.7     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA RTX A6000    On   | 00000000:65:00.0  On |                  Off |
| 43%   61C    P2    85W / 300W |  27681MiB / 49140MiB |      1%      Default |
|                               |                      |                  N/A |
+

In [79]:
import dask.array as da
from tqdm.auto import tqdm

In [10]:
images

,Array,Chunk
Bytes,20.44 GiB,31.01 MiB
Shape,"(75, 1, 6048, 6048)","(1, 1, 2016, 2016)"
Count,9450 Tasks,675 Chunks
Type,uint64,numpy.ndarray


In [117]:
mask_stack = []
for n, timepoint in tqdm(enumerate(images), total = len(images)):
    ### extract GFP channel and lowest Z plane from single time point
    gfp_z0_frame = timepoint[0,...]
    masks = segment(gfp_z0_frame)
    mask_stack.append(masks)
mask_images = da.stack(mask_stack, axis = 0) 

  0%|          | 0/75 [00:00<?, ?it/s]

In [11]:
%%time
images = images.compute().compute()

CPU times: user 1h 55min 41s, sys: 2min 47s, total: 1h 58min 28s
Wall time: 4min 23s


In [12]:
images.dtype

dtype('uint64')

In [15]:
mask_images_loaded = np.load('masks.npy',)

In [54]:
mask_images_loaded.shape

(75, 6048, 6048)

In [57]:
mask = mask_images_loaded[0]

In [56]:
viewer = napari.Viewer()
viewer.add_image(gfp, colormap = 'green', contrast_limits=[0, 3000])
viewer.add_image(rfp, colormap = 'red', blending = 'additive', contrast_limits=[100, 6000])
viewer.add_labels(mask, name = 'uncleaned')

/home/dayn/miniconda3/envs/aero/lib/python3.9/site-packages/napari_tools_menu/__init__.py:194: FutureWarning: Public access to Window.qt_viewer is deprecated and will be removed in
v0.5.0. It is considered an "implementation detail" of the napari
application, not part of the napari viewer model. If your use case
requires access to qt_viewer, please open an issue to discuss.
  self.tools_menu = ToolsMenu(self, self.qt_viewer.viewer)


<Labels layer 'uncleaned' at 0x7fb83494abb0>

# Finding the size of small masks

In [58]:
from skimage.morphology import label
from skimage.measure import regionprops

In [76]:
### small segments, need excluding
regionprops((mask == 121).astype(np.uint16))[0].area, regionprops((mask == 535).astype(np.uint16))[0].area, regionprops((mask == 48).astype(np.uint16))[0].area, regionprops((mask == 43).astype(np.uint16))[0].area

(102, 1752, 92, 282)

In [66]:
### upper bound- Too big
regionprops((mask == 2).astype(np.uint16))[0].area

3022

In [75]:
### long strips need excluding
regionprops((mask == 221).astype(np.uint16))[0].area, regionprops((mask == 61).astype(np.uint16))[0].area, regionprops((mask == 211).astype(np.uint16))[0].area

(2389, 668, 647)

# Deciding on the upper limit of 2500 area, Tidy up segmentation maps

by removing small objects

In [77]:
from skimage.morphology import remove_small_objects, remove_small_holes, binary_erosion

In [80]:
mask_images_loaded_cleaned = np.zeros(mask_images_loaded.shape, dtype = np.uint16)
for n, edited_mask in tqdm(enumerate(mask_images_loaded), total = 75):
    edited_mask = remove_small_objects(edited_mask, min_size=2500)
    mask_images_loaded_cleaned[n] = edited_mask

  0%|          | 0/75 [00:00<?, ?it/s]

# Now manually label first image

In [81]:
clean_mask = mask_images_loaded_cleaned[0]

In [92]:
viewer = napari.Viewer()
viewer.add_image(gfp, colormap = 'green', contrast_limits=[0, 3000])
viewer.add_image(rfp, colormap = 'red', blending = 'additive', contrast_limits=[100, 6000])
viewer.add_labels(clean_mask, name = 'clean')

/home/dayn/miniconda3/envs/aero/lib/python3.9/site-packages/napari_tools_menu/__init__.py:194: FutureWarning: Public access to Window.qt_viewer is deprecated and will be removed in
v0.5.0. It is considered an "implementation detail" of the napari
application, not part of the napari viewer model. If your use case
requires access to qt_viewer, please open an issue to discuss.
  self.tools_menu = ToolsMenu(self, self.qt_viewer.viewer)


<Labels layer 'clean' at 0x7fb83ec72e80>

# Make segmentation instance

In [93]:
final_mask = label(clean_mask)

In [96]:
from skimage.io import imsave
import os

In [85]:
gt_fn = f'r0{row}c0{column}f0*p*1-ch1sk1fk1fl1.tiff'
gt_fn

'r06c09f0*p*1-ch1sk1fk1fl1.tiff'

In [101]:
imsave(os.path.join('/mnt/DATA/macrohet/segmentation/ground_truth', gt_fn), final_mask.astype(np.uint16))

/tmp/ipykernel_482041/2681817267.py:1: UserWarning: /mnt/DATA/macrohet/segmentation/ground_truth/r06c09f0*p*1-ch1sk1fk1fl1.tiff is a low contrast image
  imsave(os.path.join('/mnt/DATA/macrohet/segmentation/ground_truth', gt_fn), final_mask.astype(np.uint16))


In [118]:
import sys
sys.path.append('../macrohet/')
from notify import send_sms

In [119]:
send_sms('max_proj_seg done')